In [38]:
# Module imports 
import pandas as pd
import os
import json

import dotenv

# Auto reload your development packages
%load_ext autoreload
%autoreload 2

# Load secrets and config from optional .env file
dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [23]:
# Load the datasets 

train_stats_df = pd.read_csv(os.getenv("TRAIN_STATS"))
test_stats_df = pd.read_csv(os.getenv("TEST_STATS"))
gbif_moth_checklist_df = pd.read_csv(os.getenv("GLOBAL_MOTH_CHECKLIST"))

In [24]:
# Filter rows in gbif_moth_checklist where 'acceptedTaxonKey' is present in train_stats's 'accepted_taxon_key'

gbif_moth_checklist_df = gbif_moth_checklist_df[gbif_moth_checklist_df["taxonomicStatus"]=="ACCEPTED"]
trained_gbif_moth_checklist_df = gbif_moth_checklist_df[
    gbif_moth_checklist_df['acceptedTaxonKey'].isin(train_stats_df['accepted_taxon_key'])
]

In [30]:
# Add accuracy columns to the checklist
trained_gbif_moth_checklist_df = trained_gbif_moth_checklist_df.assign(
    gbif_train_imgs = 0,
    gbif_test_imgs = 0,
    gbif_test_top1_acc = 0.0,
    gbif_test_top5_acc = 0.0,
    traps_test_imgs = 0,
    traps_test_top1_acc = 0.0,
    traps_test_top5_acc = 0.0
)


In [45]:
## Add GBIF images and accuracy number ##
checklist_df = trained_gbif_moth_checklist_df.copy()

# Add number of training images
for _, row in train_stats_df.iterrows():
    key, imgs = row["accepted_taxon_key"], row["num_gbif_train_images"]
    checklist_df.loc[checklist_df["acceptedTaxonKey"]==key, "gbif_train_imgs"] = imgs 

# Add number of test images
for _, row in test_stats_df.iterrows():
    key, imgs = row["accepted_taxon_key"], row["num_gbif_test_images"]
    checklist_df.loc[checklist_df["acceptedTaxonKey"]==key, "gbif_test_imgs"] = imgs 

# Add test accuracy
#TODO

In [46]:
## Add AMI-Traps images and accuracy number ##

# Load AMI-Traps accuracy file
with open(os.getenv("AMI_TRAPS_ACC"), "r") as file:
    ami_traps_acc = json.load(file)
ami_traps_acc_sp = ami_traps_acc["SPECIES"]

# Append the image and accuracy numbers to the checklist
for sp in ami_traps_acc_sp.keys():
    imgs = ami_traps_acc_sp[sp][2]
    top1_acc = round(ami_traps_acc_sp[sp][0]/imgs*100, 2)
    top5_acc = round(ami_traps_acc_sp[sp][1]/imgs*100, 2)
    checklist_df.loc[checklist_df["species"]==sp, ["traps_test_imgs", "traps_test_top1_acc", "traps_test_top5_acc"]] = [imgs, top1_acc, top5_acc]


In [ ]:
# Save the checklist

In [47]:
checklist_df

,taxonKey,scientificName,acceptedTaxonKey,acceptedScientificName,numberOfOccurrences,taxonRank,taxonomicStatus,family,familyKey,genus,...,species,speciesKey,iucnRedListCategory,gbif_train_imgs,gbif_test_imgs,gbif_test_top1_acc,gbif_test_top5_acc,traps_test_imgs,traps_test_top1_acc,traps_test_top5_acc
0,1731194,"Comadia henrici Grote, 1882",1731194,"Comadia henrici Grote, 1882",14,SPECIES,ACCEPTED,Cossidae,8840.0,Comadia,...,Comadia henrici,1731194.0,NE,20,5,0.0,0.0,0,0.0,0.0
2,1732027,"Anthela subfalcata (Walker, 1855)",1732027,"Anthela subfalcata (Walker, 1855)",1,SPECIES,ACCEPTED,Anthelidae,4547.0,Anthela,...,Anthela subfalcata,1732027.0,NE,5,0,0.0,0.0,0,0.0,0.0
3,1732717,"Phyllodesma americana (Harris, 1841)",1732717,"Phyllodesma americana (Harris, 1841)",5501,SPECIES,ACCEPTED,Lasiocampidae,3530.0,Phyllodesma,...,Phyllodesma americana,1732717.0,NE,621,180,0.0,0.0,1,100.0,100.0
4,1732770,"Caloecia juvenalis (Barnes & McDunnough, 1911)",1732770,"Caloecia juvenalis (Barnes & McDunnough, 1911)",6,SPECIES,ACCEPTED,Lasiocampidae,3530.0,Caloecia,...,Caloecia juvenalis,1732770.0,NE,5,1,0.0,0.0,0,0.0,0.0
5,1733172,"Kunugia magellani (de Lajonquière, 1978)",1733172,"Kunugia magellani (de Lajonquière, 1978)",3,SPECIES,ACCEPTED,Lasiocampidae,3530.0,Kunugia,...,Kunugia magellani,1733172.0,NE,5,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51879,10816146,"Cidariplura hbun Wu, Owada & Wang, 2019",10816146,"Cidariplura hbun Wu, Owada & Wang, 2019",4,SPECIES,ACCEPTED,Erebidae,4532185.0,Cidariplura,...,Cidariplura hbun,10816146.0,NE,4,0,0.0,0.0,0,0.0,0.0
51880,10829605,"Drasteria aberrans Staudinger, 1888",10829605,"Drasteria aberrans Staudinger, 1888",9,SPECIES,ACCEPTED,Erebidae,4532185.0,Drasteria,...,Drasteria aberrans,10829605.0,NE,9,2,0.0,0.0,0,0.0,0.0
51881,10904813,"Agrochola ruticilla (Esper, 1791)",10904813,"Agrochola ruticilla (Esper, 1791)",641,SPECIES,ACCEPTED,Noctuidae,7015.0,Agrochola,...,Agrochola ruticilla,10904813.0,NE,626,186,0.0,0.0,0,0.0,0.0
51882,11420352,Thysanoplusia lectula (Walker),11420352,Thysanoplusia lectula (Walker),20,SPECIES,ACCEPTED,Noctuidae,7015.0,Thysanoplusia,...,Thysanoplusia lectula,11420352.0,NE,15,5,0.0,0.0,0,0.0,0.0
